<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>From local Spark MLlib model to cloud with watson-machine-learning-client</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://www-03.ibm.com/services/learning/images/journey_watson_wat_machine_learning_600x280.jpg" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to work with **``watson-machine-learning-client``** library. This notebook introduces commands for getting data, model persistance to Watson Machine Learning repository, model deployment and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3 and Apache® Spark 2.0.

You will use publicly available data "GoSales Transactions for Naive Bayes Model" about anonymous outdoor equipment purchase to predict clients' interests in terms of product line like: golf accessories, camping equipement and others.


## Learning goals

The learning goals of this notebook are:
- Load and explore data
- Prepare data for training and evaluation
- Create spark ml pipeline
- Train and evaluate model
- Persist local model in Watson Machine Learning repository using client library
- Deploy model for online scoring using client library
- Score sample records using client library


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create an Apache® Spark machine learning model](#model)
4.	[Persist created model using api client](#persistance)
5.	[Deploy and score in a Cloud using api client](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Make sure that you are using a Spark 2.0 kernel.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

In [1]:
!pip install wget --user --upgrade

Requirement already up-to-date: wget in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages


In [2]:
import wget
import json
import os

In [3]:
filename = 'GoSales_Tx_NaiveBayes.csv'

if not os.path.isfile(filename):
    link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
    filename = wget.download(link_to_data)

print(filename)

GoSales_Tx_NaiveBayes.csv


The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [4]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [5]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [6]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [7]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [8]:
splitted_data = df_data.randomSplit([0.8, 0.2], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 48176
Number of testing records : 12076


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [9]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [10]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [11]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [12]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [13]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [14]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [15]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [16]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.584796


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist created model with ``watson-machine-learning-client``

In this section you will learn how to store your model in Watson Machine Learning repository by using common python client.

#### First, you must install client libraries from pypi

In [17]:
!pip install watson-machine-learning-client --user --upgrade

    100% |████████████████████████████████| 307kB 1.6MB/s eta 0:00:01
Requirement already up-to-date: lomond in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tabulate in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tqdm in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: requests in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-pack

Import installed client by running below code.

In [18]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**You can check sphinx documentation for watson-machine-learning-client either by visiting [documentation webpage](https://wml-api-pyclient.mybluemix.net) or running below code.**

In [19]:
from IPython.display import IFrame
IFrame('https://wml-api-pyclient.mybluemix.net', width='100%', height=400)

### 4.1: Work with your instance

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [20]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "*****",
  "username": "*****",
  "password": "*****",
  "instance_id": "*****"
}

**Tip**: Credentials can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

#### Create API client by running below code.

In [21]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [22]:
instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

{
  "metadata": {
    "created_at": "2017-06-26T11:49:37.500Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f",
    "modified_at": "2017-12-05T13:53:52.454Z",
    "guid": "a153b37a-2b65-47e6-b0ef-2d71da079c4f"
  },
  "entity": {
    "region": "us-south",
    "account": {
      "id": "8dba8395f5b8dc6fcab49d43161ab9f1",
      "type": "TRIAL",
      "name": "Wojciech Sobala's Account"
    },
    "deployments": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/deployments"
    },
    "source": "Bluemix",
    "status": "Active",
    "plan_id": "3f6acf43-ede8-413a-ac69-f8af3bb0cbfe",
    "published_models": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models"
    },
    "organization_guid": "a7b3ef43-5dc2-41fc-8fa7-29b0846e5807",
    "plan": "lite",
    "space_guid": "a6b628f9-3f64-45c0-bf38-739876b63fec",
    "

### 4.2: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

In [23]:
model_props = {"authorName":"IBM", "authorEmail":"ibm@ibm.com"}
published_model = client.repository.store_model(model_rf, "LOCALLY created Product Line Prediction model", meta_props=model_props, training_data=train_data)

### 4.3: Get model details

In [24]:
model_details = client.repository.get_details(published_model.uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "created_at": "2017-12-05T14:02:20.939Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/ac94d983-8cfe-43a1-af8e-c02352c685c8",
    "modified_at": "2017-12-05T14:02:24.119Z",
    "guid": "ac94d983-8cfe-43a1-af8e-c02352c685c8"
  },
  "entity": {
    "latest_version": {
      "created_at": "2017-12-05T14:02:24.119Z",
      "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/ac94d983-8cfe-43a1-af8e-c02352c685c8/versions/c28564d2-95bf-4215-a291-6805f87d4141",
      "guid": "c28564d2-95bf-4215-a291-6805f87d4141"
    },
    "runtime_environment": "None Provided",
    "name": "LOCALLY created Product Line Prediction model",
    "learning_configuration_url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/ac94d983-8cfe-43a1-af8e-c02352c685c8/learning_configuration",
    "model_type": "mllib-2.1",
    "feedback_url": "https://ibm-w

#### Get all models

In [25]:
models_details = client.repository.get_details()

print(json.dumps(models_details, indent=2))

{
  "resources": [
    {
      "metadata": {
        "created_at": "2017-12-05T13:30:42.254Z",
        "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699",
        "modified_at": "2017-12-05T13:31:19.354Z",
        "guid": "4ab4b831-8bf5-4d19-8231-896bf1f2f699"
      },
      "entity": {
        "label_col": "",
        "latest_version": {
          "created_at": "2017-12-05T13:30:44.295Z",
          "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/versions/7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc",
          "guid": "7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc"
        },
        "runtime_environment": "python-3.5",
        "deployments": {
          "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/deployments",
          "count": 1
        },


### 4.3: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [26]:
loaded_model = client.repository.load(published_model.uid)

You can pass test_data to loaded model ``transform()`` method to make sure that model has been loaded correctly.

In [27]:
test_predictions = loaded_model.transform(test_data)

In [28]:
test_predictions.show(3)

+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS| PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 18|        Single|      Other|  0.0|          0.0|      1.0|              1.0|[1.0,18.0,1.0,0.0]|[5.40697857263576...|[0.27034892863178...|       1.0|Personal Accessories|
|Camping Equipment|     F| 18|        Single|     Retail|  0.0|          7.0|      1.0|              1.0|[1.0,18.0,1.0,7.0]|[3.01171000473832...|[0.15058550023691...|       1.0|Personal Accessories|
|Camp

As you can see the loaded model works. You have already learned how save and load the model from Watson Machine Learning repository.

### 4.4: Delete model

You can delete published model from Watson Machine Learning repository using below code. The code is commented out at this stage since the model will be needed later for deployment.

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client. 

### 5.1: Create online deployment

In [32]:
created_deployment = client.deployments.create(published_model.uid, "Deployment of locally created model XXX")

Now you can print an online scoring endpoint. 

In [33]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/ac94d983-8cfe-43a1-af8e-c02352c685c8/deployments/e1260b2c-043f-40f9-967a-776d02a1de29/online


### 5.2: Get deployments

In [34]:
deployments = client.deployments.get_details()

In [35]:
print(json.dumps(deployments, indent=2))

{
  "resources": [
    {
      "metadata": {
        "created_at": "2017-12-05T13:31:18.356Z",
        "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/deployments/980ed6fb-c16f-4d98-8c71-d6eeb95093fe",
        "modified_at": "2017-12-05T13:31:19.322Z",
        "guid": "980ed6fb-c16f-4d98-8c71-d6eeb95093fe"
      },
      "entity": {
        "scoring_url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/deployments/980ed6fb-c16f-4d98-8c71-d6eeb95093fe/online",
        "runtime_environment": "python-3.5",
        "published_model": {
          "created_at": "2017-12-05T13:31:18.312Z",
          "name": "My cool mnist model",
          "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699",
    

You can get deployment_url by parsing deployment details.

In [36]:
deployment_url = client.deployments.get_deployment_url(created_deployment)

print(deployment_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/ac94d983-8cfe-43a1-af8e-c02352c685c8/deployments/e1260b2c-043f-40f9-967a-776d02a1de29


### 5.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [37]:
scoring_payload = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}

Use ``client.deployments.score()`` method to run scoring.

In [38]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [39]:
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "GENDER",
    "AGE",
    "MARITAL_STATUS",
    "PROFESSION",
    "PRODUCT_LINE",
    "label",
    "PROFESSION_IX",
    "GENDER_IX",
    "MARITAL_STATUS_IX",
    "features",
    "rawPrediction",
    "probability",
    "prediction",
    "predictedLabel"
  ],
  "values": [
    [
      "M",
      23,
      "Single",
      "Student",
      "Camping Equipment",
      0.0,
      6.0,
      0.0,
      1.0,
      [
        0.0,
        23.0,
        1.0,
        6.0
      ],
      [
        5.258712981131878,
        6.723421733525897,
        6.0132018750333724,
        0.218060296213069,
        1.7866031140957863
      ],
      [
        0.26293564905659383,
        0.3361710866762948,
        0.3006600937516686,
        0.010903014810653448,
        0.0893301557047893
      ],
      1.0,
      "Personal Accessories"
    ],
    [
      "M",
      55,
      "Single",
      "Executive",
      "Camping Equipment",
      0.0,
      3.0,
      0.0,
      1.0,
      [
        0

### 5.4: Delete deployment

Use the following method to delete deployment.

In [40]:
client.deployments.delete(deployment_url)

True

You can check number of depoyments by running ``client.deployment.get()`` method.

In [41]:
client.deployments.get_details()

{'count': 2,
 'resources': [{'entity': {'deployed_version': {'guid': '7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc',
     'url': 'https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/versions/7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc'},
    'description': 'Model deployment',
    'model_type': 'tensorflow-1.2',
    'name': 'Mnist model deployment',
    'published_model': {'created_at': '2017-12-05T13:31:18.312Z',
     'guid': '4ab4b831-8bf5-4d19-8231-896bf1f2f699',
     'name': 'My cool mnist model',
     'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699'},
    'runtime_environment': 'python-3.5',
    'scoring_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/deployments/980ed6fb-c16f-4d98-8c71-d6eeb95093fe/online',
    'status': 'ACTIVE',
    'type': 'online'}

### 5.5: Delete model

In [42]:
client.repository.delete(published_model.uid)

You can check number of your models by running below ``client.repository.get_details()`` method.

In [43]:
client.repository.get_details()

{'count': 1,
 'resources': [{'entity': {'deployed_version': {'guid': '7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc',
     'url': 'https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/versions/7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc'},
    'deployments': {'count': 1,
     'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/deployments'},
    'evaluation_metrics_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/evaluation_metrics',
    'feedback_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/a153b37a-2b65-47e6-b0ef-2d71da079c4f/published_models/4ab4b831-8bf5-4d19-8231-896bf1f2f699/feedback',
    'label_col': '',
    'latest_version': {'created_at': '2017-12-05T13:30:44.295Z',
     'guid': '7cfb5dcc-6b62-419e-a1db-7e7d7dc8f0cc',
     'url': 'https://

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.